In [9]:
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
from sklearn.ensemble import RandomForestClassifier

class MultiNomialNB:
    def __init__(self):
        self.thetak = defaultdict(float)
        # self.thetajk = defaultdict(partial(np.ndarray, 0))
        self.thetajk= []
        self.classes = []
        
    def fit(self, X, y):
        self.classes = y.unique()
        self.thetajk = np.arange(len(self.classes)*X.shape[1], dtype=float).reshape(len(self.classes),X.shape[1])

        for i,k in enumerate(self.classes):
            self.thetak[k] = (y==k).sum()/float(y.shape[0])
            indices = np.where(y==k)[0]
            filteredX = X[indices]
            self.thetajk[i] = (filteredX.sum(axis=0)+0.01)/float(filteredX.shape[0]+0.02)
            self.thetajk[i] = self.thetajk[i]
        print(self.thetak, self.thetajk)
    
    def predict(self, X):
        left = X.dot(np.log(self.thetajk).T)
        right = (1-X.toarray()).dot(np.log(1-self.thetajk).T)
        temp_sum = np.add(left, right)
        max_num = np.argmax(temp_sum,axis=1)
        return self.classes[max_num]

    def score(self, X, y):
        y_pred = self.predict(X)
        
        score = (y_pred==y).sum()/float(y.shape[0])
        return score

if __name__ == "__main__":
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
    from sklearn import preprocessing

    redditData = pd.read_csv("../data/reddit_train.csv") #, sep="\n", header=None) 
    redditDataTest = pd.read_csv("../data/reddit_test.csv") # sep="\n", header=None)

    comments = redditData.iloc[:,1]
    subreddits = redditData.iloc[:,-1]
    comments = [str (item) for item in comments]
    
    tf = TfidfVectorizer(max_features=1000, stop_words='english')
    comments = tf.fit_transform(comments).toarray()
    le = preprocessing.LabelEncoder()
    subreddits = le.fit_transform(subreddits)
    print(comments.shape)
    print(subreddits)
    x_train, x_test, y_train, y_test = train_test_split(comments, subreddits, test_size=0.2, random_state=4)
    
    print("ready")
#     classifier = RandomForestClassifier(n_estimators=10, random_state=0)
#     classifier.fit(x_train, y_train)
#     classifier.score(x_test, y_test)
    
    from keras.models import Sequential
    
    model = Sequential()
    model.add(Embedding(1000, 100, input_length=x_train.shape[1]))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(13, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    epochs = 5
    batch_size = 64

    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
    
    accr = model.evaluate(X_test,Y_test)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
    cmultiNB = MultiNomialNB()
    cmultiNB.fit(x_train_v, y_train)

#     print(cmultiNB.score(x_test_v, y_test))

(70000, 1000)
[11 14 12 ...  6  4  1]
ready


ModuleNotFoundError: No module named 'keras'

In [4]:
print(classifier.score(x_test, y_test))

0.2681428571428571


In [4]:
!pip uninstall keras y

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
^C
Operation cancelled by user
